Thanks to https://habr.com/ru/post/467081/

In [44]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np
import time
import os
import re
import glob

# Настройки

In [10]:
# Нужны реальные, чтобы не банили (смотрите свои в разделе network в инструментах разработчика браузера)
headers = {
    'User-Agent':'???',
    'Accept-language' : "*/*",
    'Accept' : "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
}

# Имитация живого пользователя
delays = [5, 6, 5.5, 5.7, 6.1, 6.3, 5.8, 4, 4.5]

In [25]:
def get_top500_film_ids():
    """
    Выкачивает с кинопоиска id фильмов из рейтинга топ 500
    
    Return:
     list (str): лист id в текстовом формате 
    """
    ids = []
    # На одной странице там 25 фильмов
    for page in tqdm(range(1,21)):
        url = f"https://www.kinopoisk.ru/top/lists/1/filtr/all/sort/order/page/{page}/"
        r = requests.get(url, headers = headers) # отправка http запроса
        soup = BeautifulSoup(r.text, 'html.parser')# создание html парсера
        posters = soup.find_all(class_="poster")
        for poster in posters:
            id_ = poster.find('div').get('data-film-id')
            ids.append(id_)  
        time.sleep(np.random.choice(delays))
    return ids

In [26]:
def load_reviews(url):
    """
    Загружает все текстовые отзывы с url в текстовом формате
    
    Args:
        url (str): Адрес страницы с отзывами на кинопоиске
        
    Return:
        list (str): Список текстовых отзывов (отзыв помещается на одной строке)
    """
    # отправка http запроса
    r = requests.get(url, headers = headers)
    # создание html парсера
    soup = BeautifulSoup(r.text, 'html.parser')
    # сохранение только отзывов
    reviews = soup.find_all(class_='_reachbanner_')
    review_converted = []
    for review in reviews:
        # очистка от лишней html разметки
        review = review.find_all(text=True)
        # приведение к строке всех найденных элементов
        for i in review:
            map(str, i)
        # соединение частей отзыва в единую строку
        review = ' '.join(review)
        review=re.sub("\n"," ",review)
        review=re.sub("\r"," ",review)
        review_converted.append(review)
    return review_converted

In [27]:
def get_name(url): 
    """
    Получение имени фильма
    
    Args: 
        url (str): Адрес фильма
        
    Return:
        str: Название фильма
    """
    r = requests.get(url, headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    name = soup.find(class_='alternativeHeadline')
    name_clean = name.find_all(text = True)
    # Сохранение первого элемента, т. к. извлекается также год фильма
    return str(name_clean[0]) 

In [40]:
def parsing(film_url, status):
    """
    Загружаем все отзывы о фильме в определенной тональности
    
    Args:
        film_url (str): Адрес фильма
        status (str): Тип тональности (доступно good, bad, neutral)
    
    Returns:
        list(str): Массив текстовых отзывов (каждый отзыв не содержит переноса строки)
    """
    page = 1
    all_reviews = []
    # Выбор рандомной задержки
    time.sleep(np.random.choice(delays))
    # Перебираем все страницы с отзывами к фильму
    while True:
        reviews = load_reviews(film_url + f'reviews/ord/rating/status/{status}/perpage/200/page/{page}/')
        if reviews == []:
            break
        else:
            all_reviews += reviews
            page += 1
            # Выбор рандомной задержки
            time.sleep(np.random.choice(delays))
            
    return all_reviews

In [41]:
def load_film_reviews(film_ids, statuses=['good', 'bad'], dir_='reviews', overwrite=False):
    """
    Для указанных индексов фильмов загружает в файлы отзывы в такой строктуре по файлам:
    {status}/{film_id.txt} - будут содержаться все отзывы нужного статуса по конкретному фильму
    
    Args:
        film_ids (list(str)): Идентетификаторы фильмов для сохранения отзывов
        statuses (list(str)): Статусы отзывов для сохранения
        dir_ (str): Имя директории для сохранения
        overwrite (bool): Если True, то перезаписывать существующие файлы
    """
    
    # Eсли папок для сохранения не существует то они создадутся
    for status in statuses:
        if not os.path.exists(os.path.join(dir_, status)):
            os.makedirs(os.path.join(dir_, status))
    
    for film_id in tqdm(film_ids):  
        url = f'https://www.kinopoisk.ru/film/{film_id}/'
        for status in statuses:
            # Путь для сохранения
            path_save = os.path.join(dir_, status, f'{film_id}.txt')
            # Сначала проверяем - скачивали мы этот фильм или нет
            if os.path.exists(path_save) and not overwrite:
                continue
            
            try:
                # Загружаем отзывы по фильмам
                reviews = parsing(film_url=url, status=status)
                # Сохраняем их
                with open(path_save, 'w', encoding='utf-8') as filesave:
                    filesave.write(' \n'.join(reviews))
                
            # Во время бана будет получена ошибка AttributeError
            except AttributeError:
                print('Бан получен: {}, {}'.format(url, status))
                break
        # Если цикл не прерывался то продолжаем
        else:
            continue
        # Выходим из цикла, если прерывалось
        break

In [47]:
def combine_txt_files(dir_, path):
    """
    Соединяет все текстовые файлы в директории в один единый
    
    Args:
        dir_ (str): Директория для которой нужно выполнить операцию
        path (str): Название итогового файла с объединением
    """
    file_mask = os.path.join(dir_, "*.txt")
    # Находим все подходящие файлы
    files = glob.glob(file_mask)
    with open(path, 'w', encoding='utf-8') as f:
        for file in files:
            with open(file, 'r',  encoding='utf-8') as fr:
                f.write(fr.read())
                # Добавим перенос строки
                f.write('\n')

# Загружаем id топовые фильмы

In [18]:
%%time
top_films_ids = get_top500_film_ids()


Wall time: 2min 7s


In [19]:
len(top_films_ids)

500

Save films_id

In [22]:
with open('top_films_ids.txt', 'w') as f:
    f.write(','.join(top_films_ids))

# Загружаем отзывы по категориям

In [42]:
film_ids_to_load = top_films_ids[:2]

In [43]:
%%time
load_film_reviews(film_ids=film_ids_to_load)


Wall time: 1min 16s


# Объединяем отзывы в единый файл

In [48]:
%%time
combine_txt_files('reviews/good', 'good_reviews.txt')

Wall time: 133 ms


In [49]:
%%time
combine_txt_files('reviews/bad', 'bad_reviews.txt')

Wall time: 18 ms
